In [9]:
import numpy as np
import numpy.random as rd

import SALib.sample.saltelli as saltelli
import SALib.analyze.sobol as sobol
import SALib as sa

import matplotlib.pyplot as plt

# Monte Carlo simulations for total index

In [10]:
def jansen_estimator(A,B,f):
    #i - total index for i-th element computed by Jansen estimator
    N = A.shape[0] # number of samples
    Vt = np.zeros(n_vars)
    for i in range(n_vars):
        A_B = np.hstack((A[:,:i],B[:,i].reshape(N,1),A[:,i+1:]))
        for j in range(N):
            Vt[i] = Vt[i] + ( f(A[j])-f(A_B[j]) )**2
    Vt = Vt/2/N
    return Vt

# Test case 1: Sobol G function

In [11]:
#Different versions of the Sobol function
def sobol_g_star_prod(a,alpha,delta,x):
    n = len(a)
    sobol_g_star = 1
    xdelta = [x[i]+delta[i] - int(x[i]+delta[i]) for i in range(n)]
    for i in range(n):
        sobol_g_star = sobol_g_star * ((1+alpha[i])*abs(2*xdelta[i]-1)**alpha[i] + a[i])/(1+a[i])
    return sobol_g_star

def sobol_g_prod(a,x):
    n = a.shape[0]
    sobol_g = 1
    for i in range(n):
        sobol_g = sobol_g * (abs(4*x[i]-2)+a[i])/(1+a[i])   
    return sobol_g

#This one is for testing
def simple_g(a):
    g = 1
    for i in range(len(a)):
        g = g * (2+a[i])/(1+a[i])    
    return g

#stress test to check implementation of sobol_g_star_prod
#when alpha = 1 and delta = 0, sobol_g_star_prod becomes sobol_g_prod
# while True:
#     a = rd.rand(n_vars)*10
#     x = rd.rand(n_vars)
#     res1 = sobol_g_star_prod(a,alpha,x)
#     res2 = sobol_g_prod(a,x)
#     print(str(res1)+' '+str(res2)+' all good')
#     if res1 != res2:
#         print('not correct')
#         print(a)
#         print(x)
#         break

In [12]:
N = 200 #number of samples in MC

#Specify function parameters
n_vars = 10
delta = rd.rand(n_vars)

#Choose case for a and alpha
case = 1

if case==1:
    alpha = np.ones(n_vars)
    a = [0,0,9,9,9,9,9,9,9,9]
elif case==2:
    alpha = np.ones(n_vars)
    a = [0,0.1,0.2,0.3,0.4,0.8,1,2,3,4]
elif case==3:
    alpha = 0.5*np.ones(n_vars)
    a = [0,0,9,9,9,9,9,9,9,9]
elif case==4:
    alpha = 0.5*np.ones(n_vars)
    a = [0,0.1,0.2,0.3,0.4,0.8,1,2,3,4]
elif case==5:
    alpha = 2*np.ones(n_vars)
    a = [0,0,9,9,9,9,9,9,9,9]
elif case==6:
    alpha = 2*np.ones(n_vars)
    a = [0,0.1,0.2,0.3,0.4,0.8,1,2,3,4]


def f(x):
    return sobol_g_star_prod(a,alpha,delta,x)

### 1. Variances from analytical solution

In [13]:
def v_analytical(a,alpha):
    return [ alpha[i]**2/(1+2*alpha[i])/(1+a[i])**2  for i in range(len(a))]

Vt_an = v_analytical(a,alpha)

### 2. Variances from MC simulations

In [14]:
samples = sa.sample.sobol_sequence.sample(N,2*n_vars)
A,B = samples[:,:n_vars],samples[:,n_vars:] 

Vt_mc = jansen_estimator(A,B,f)
y_mc = np.array([f(samples[i]) for i in range(samples.shape[0])])
len(y_mc)

200

### 3. SALib results

In [ ]:
N = 2000
names  = [_+str(i) for _ in ['a'] for i in range(n_vars)]
bounds = [[0,1] for _ in range(n_vars)]

problem = {
    'num_vars': n_vars,
    'names':    names,
    'bounds':   bounds
}

samples = saltelli.sample(problem, N, calc_second_order=False)
y_sa = np.array([f(samples[i]) for i in range(samples.shape[0])])
sa_results = sobol.analyze(problem, y, calc_second_order=False)
len(y_sa)

In [17]:
sobol.

### 4. Print variances for comparison

In [ ]:
#TODO should be divided by output variance
Vt_an = [Vt_an[i]/sum(Vt_an) for i in range(n_vars)]
Vt_mc = [Vt_mc[i]/np.var(y_mc) for i in range(n_vars)]
Vt_sa = [Vt_sa[i]/np.var(y_sa) for i in range(n_vars)]

print(["{0:0.5f}".format(i) for i in Vt_an])
print(["{0:0.5f}".format(i) for i in Vt_mc])
print(["{0:0.5f}".format(i) for i in Vt_sa])

# Test case 2: K function

In [ ]:
def k_function(x):
    k = len(x)
    k_func = 0
    for i in range(1,k+1):
        print(i)
        k_func = k_func + (-1)**i*np.prod(x[:i])
    return k_func

In [ ]:
def